#### Plot splice sites logo

In [1]:
import pandas as pd

cosi_long = pd.read_pickle("../data/processed_files/cosi_long.pkl.gz")

c3 = cosi_long[cosi_long.coSI=='$\\theta_3$'].copy()
c3["pos"] = c3["end"]+1

c5 = cosi_long[cosi_long.coSI=='$\\theta_5$'].copy()
c5["pos"] = c5["start"]

Create BED files for HBV splice sites

In [2]:
mc5 = c5[(c5["HBV_genotype"].str.contains("B|C"))].groupby("pos").max("Splicing efficiency")
mc5["start"] = mc5["start"]-10
mc5["end"] = mc5["end"]+18
mc5.to_csv("../data/processed_files/mc5.bed", sep="\t", header=None, index=None)

mc3 = c3[(c3["HBV_genotype"].str.contains("B|C"))].groupby("pos").max("Splicing efficiency")
mc3["start"] = mc3["start"]-20
mc3["end"] = mc3["end"]+8
mc3.to_csv("../data/processed_files/mc3.bed", sep="\t", header=None, index=None)

Get fasta sequences for HBV splice sites 

In [ ]:
# select genotypes B, C, and F, and recombinant forms (RF)
! awk 'BEGIN{RS=">"}NR>1{sub("\n","\t"); gsub("\n",""); print RS$0}' ../ref/hbvdb/pgrna/pgrna.fa \
    | awk '(/B.fa|C.fa|F.fa|RF.fa/) && (length($NF)==3240) {print $1 "|" $2, "\t" $NF}' \
    | sed 's/\.fa//' | awk '{print $1 "\n" $NF}' > ../data/processed_files/hbvdb_pgrna.fa

# get sequences and convert the format to BED
! grep ">" ../data/processed_files/hbvdb_pgrna.fa | sed 's/>//' \
    | while read i; do awk -v chrom=$i '{print chrom "\t" $0}' ../data/processed_files/mc5.bed; done \
    | bedtools getfasta -bed stdin -fi ../data/processed_files/hbvdb_pgrna.fa -tab -name \
    | sed 's/::/\t/;s/|/\t/3;s/:/\t/;s/-/\t/' \
    | awk 'BEGIN{OFS="\t"} {print $2,$4,$5,$3,$1,$NF}' > ../data/processed_files/hbvdb_pgrna_mc5.bed

! grep ">" ../data/processed_files/hbvdb_pgrna.fa | sed 's/>//' \
    | while read i; do awk -v chrom=$i '{print chrom "\t" $0}' ../data/processed_files/mc3.bed; done \
    | bedtools getfasta -bed stdin -fi ../data/processed_files/hbvdb_pgrna.fa -tab -name \
    | sed 's/::/\t/;s/|/\t/3;s/:/\t/;s/-/\t/' \
    | awk 'BEGIN{OFS="\t"} {print $2,$4,$5,$3,$1,$NF}' > ../data/processed_files/hbvdb_pgrna_mc3.bed

Plot sequence logos for HBV splice sites

In [ ]:
# randomly select overlaps for sliding windows
! SEED=123 && \
  MIN=1 && \
  MAX=12 && \
  \
  RANDOM=$SEED && \
  for i in {1..5}; do \
    RAND_NUM=$(( ( RANDOM % (MAX - MIN + 1) ) + MIN )) && \
    pyfasta split -k30 -n1 -o $RAND_NUM ../data/processed_files/hbvdb_pgrna.fa; done

! cat ../data/processed_files/hbvdb_pgrna.split.30mer.*overlap.fa > ../data/processed_files/hbvdb_pgrna.split.30mer.fa

# randomly sample 10% sequences
SEQS=`grep -c ">" ../data/processed_files/hbvdb_pgrna.split.30mer.fa | awk '{print int(($1*0.01)+0.5)}'`
! pyfastx sample -n ${SEQS} -s 42 --sequential-read -o ../data/processed_files/hbvdb_pgrna.split.30mer.10perc.fa  ../data/processed_files/hbvdb_pgrna.split.30mer.fa

# separate higher usage splice sites from lower usage and get fasta sequences
! awk '$5>=0.1 {print ">" $1 "\n" $NF}' ../data/processed_files/hbvdb_pgrna_mc5.bed > ../data/processed_files/hbvdb_pgrna_mc5_high.fa
! awk '$5<0.1 {print ">" $1 "\n" $NF}' ../data/processed_files/hbvdb_pgrna_mc5.bed > ../data/processed_files/hbvdb_pgrna_mc5_low.fa

! awk '$5>=0.1 {print ">" $1 "\n" $NF}' ../data/processed_files/hbvdb_pgrna_mc3.bed > ../data/processed_files/hbvdb_pgrna_mc3_high.fa
! awk '$5<0.1 {print ">" $1 "\n" $NF}' ../data/processed_files/hbvdb_pgrna_mc3.bed > ../data/processed_files/hbvdb_pgrna_mc3_low.fa

# plot logo
! for i in ../data/processed_files/hbvdb_pgrna_mc*fa; do \
    mkdir -p ${i%.*} && \
    kpLogo $i -bgfile ../data/processed_files/hbvdb_pgrna.split.30mer.10perc.fa -o ${i%.*}/kpLogo; done

Create BED files for human CDS splice sites

In [ ]:
# Filter out splice sites in the UTRs
bedtools intersect -a ../data/processed_files/ss.bed -b <(bed12ToBed6 -i ../ref/gencode.v41lift37.annotation.bed) -wo -s \
    | bedtools intersect -a stdin -b ../ref/utr.hs.bed -v \
    | cut -f-6 | awk '!seen[$0]++' \
    | bedtools intersect -a stdin -b <(awk 'BEGIN{OFS="\t"} {print $1,$2+1,$3,$4,$5,$6}' ../data/huh7/intron.zeta.pUC57.bed) -wo \
    | awk 'BEGIN{OFS="\t"} ($2==$8) && /donor/ {print $1,$2-10,$3+18,$5 "|" $11,$4,$6}' \
    | bedtools getfasta -bed stdin -fi ../ref/hg19.fa -tab -name -s \
    | sed 's/::/\t/;s/|/\t/;s/:/\t/;s/-/\t/;s/(/\t/;s/)//' \
    | awk 'BEGIN{OFS="\t"} {print $3,$4,$5,$1,$2,$6,toupper($NF)}' \
    | sort -u > ../data/processed_files/exonicss_mc5.bed

bedtools intersect -a ../data/processed_files/ss.bed -b <(bed12ToBed6 -i ../ref/gencode.v41lift37.annotation.bed) -wo -s \
    | bedtools intersect -a stdin -b ../ref/utr.hs.bed -v \
    | cut -f-6 | awk '!seen[$0]++' \
    | bedtools intersect -a stdin -b <(awk 'BEGIN{OFS="\t"} {print $1,$2,$3-1,$4,$5,$6}' ../data/huh7/intron.zeta.pUC57.bed) -wo \
    | awk 'BEGIN{OFS="\t"} ($3==$9) && /acceptor/ {print $1,$2-20,$3+8,$5 "|" $11,$4,$6}' \
    | bedtools getfasta -bed stdin -fi ../ref/hg19.fa -tab -name -s \
    | sed 's/::/\t/;s/|/\t/;s/:/\t/;s/-/\t/;s/(/\t/;s/)//' \
    | awk 'BEGIN{OFS="\t"} {print $3,$4,$5,$1,$2,$6,toupper($NF)}' \
    | sort -u > ../data/processed_files/exonicss_mc3.bed

# get genes with CDS splice sites
cat ../data/processed_files/exonicss_mc*.bed \
    | cut -f4 | sort -u \
    | join -1 1 -2 12 -o 2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,2.10,2.11,2.12,2.13,2.14,2.15 -t$'\t' - \
    <(zcat ../ref/gencode.v41lift37.annotation.genepred.txt.gz | sort -k12,12) \
    | genePredToBed stdin stdout \
    | bedtools getfasta -bed stdin -fi ../ref/hg19.fa -split -name -s > ../data/processed_files/exonicss_genes.fa

Plot sequence logos for human CDS splice sites

In [ ]:
# randomly select overlaps for sliding windows
! SEED=123 && \
  MIN=1 && \
  MAX=12 && \
  \
  RANDOM=$SEED && \
  for i in {1..5}; do \
    RAND_NUM=$(( ( RANDOM % (MAX - MIN + 1) ) + MIN )) && \
    pyfasta split -k30 -n1 -o $RAND_NUM ../data/processed_files/exonicss_genes.fa; done
    
! cat ../data/processed_files/exonicss_genes.split.30mer.*overlap.fa > ../data/processed_files/exonicss_genes.split.30mer.fa

# randomly sample 1% sequences
SEQS=`grep -c ">" ../data/processed_files/exonicss_genes.split.30mer.fa | awk '{print int(($1*0.01)+0.5)}'`
! pyfastx sample -n ${SEQS} -s 42 --sequential-read -o ../data/processed_files/exonicss_genes.split.30mer.1perc.fa  ../data/processed_files/exonicss_genes.split.30mer.fa

# get fasta sequences
! awk '{print ">" $1 "\n" $NF}' ../data/processed_files/exonicss_mc5.bed > ../data/processed_files/exonicss_mc5.fa
! awk '{print ">" $1 "\n" $NF}' ../data/processed_files/exonicss_mc3.bed > ../data/processed_files/exonicss_mc3.fa

# plot logo
! for i in ../data/processed_files/exonicss_mc*fa; do \
    mkdir -p ${i%.*} && \
    kpLogo $i -bgfile ../data/processed_files/exonicss_genes.split.30mer.1perc.fa -o ${i%.*}/kpLogo; done

Create BED files for human typical splice sites

In [ ]:
! bedtools intersect -a ../data/processed_files/ss.bed -b <(awk 'BEGIN{OFS="\t"} {print $1,$2+1,$3,$4,$5,$6}' ../data/huh7/intron.zeta.pUC57.bed) -wo \
    | awk 'BEGIN{OFS="\t"} ($2==$8) && /donor/ {print $1,$2-10,$3+18,$5 "|" $11,$4,$6}' \
    | bedtools getfasta -bed stdin -fi ../ref/hg19.fa -tab -name -s \
    | sed 's/::/\t/;s/|/\t/;s/:/\t/;s/-/\t/;s/(/\t/;s/)//' \
    | awk 'BEGIN{OFS="\t"} {print $3,$4,$5,$1,$2,$6,toupper($NF)}' \
    | sort -u > ../data/processed_files/hg19_mc5.bed
# exclude CDS splice donor sites
! comm -23 <(sort ../data/processed_files/hg19_mc5.bed) <(sort ../data/processed_files/exonicss_mc5.bed) > ../data/processed_files/hg19_mc5.bed2
! mv ../data/processed_files/hg19_mc5.bed2 ../data/processed_files/hg19_mc5.bed

! bedtools intersect -a ../data/processed_files/ss.bed -b <(awk 'BEGIN{OFS="\t"} {print $1,$2,$3-1,$4,$5,$6}' ../data/huh7/intron.zeta.pUC57.bed) -wo \
    | awk 'BEGIN{OFS="\t"} ($3==$9) && /acceptor/ {print $1,$2-20,$3+8,$5 "|" $11,$4,$6}' \
    | bedtools getfasta -bed stdin -fi ../ref/hg19.fa -tab -name -s \
    | sed 's/::/\t/;s/|/\t/;s/:/\t/;s/-/\t/;s/(/\t/;s/)//' \
    | awk 'BEGIN{OFS="\t"} {print $3,$4,$5,$1,$2,$6,toupper($NF)}' \
    | sort -u > ../data/processed_files/hg19_mc3.bed
# exclude CDS splice acceptor sites
! comm -23 <(sort ../data/processed_files/hg19_mc3.bed) <(sort ../data/processed_files/exonicss_mc3.bed) > ../data/processed_files/hg19_mc3.bed2
! mv ../data/processed_files/hg19_mc3.bed2 ../data/processed_files/hg19_mc3.bed

# get genes with typical splice sites
! cat ../data/processed_files/hg19_mc*.bed \
    | cut -f4 | sort -u \
    | join -1 1 -2 12 -o 2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,2.10,2.11,2.12,2.13,2.14,2.15 -t$'\t' - \
    <(zcat ../ref/gencode.v41lift37.annotation.genepred.txt.gz | sort -k12,12) \
    | genePredToBed stdin stdout \
    | bedtools getfasta -bed stdin -fi ../ref/hg19.fa -split -name -s > ../ref/hg19_genes.fa

Plot sequence logos for human typical splice sites

In [ ]:
# randomly select overlaps for sliding windows
! SEED=123 && \
  MIN=1 && \
  MAX=12 && \
  \
  RANDOM=$SEED && \
  for i in {1..5}; do \
    RAND_NUM=$(( ( RANDOM % (MAX - MIN + 1) ) + MIN )) && \
    pyfasta split -k30 -n1 -o $RAND_NUM ../data/processed_files/hg19_genes.fa; done
    
! cat ../data/processed_files/hg19_genes.split.30mer.*overlap.fa > ../data/processed_files/hg19_genes.split.30mer.fa

# randomly sample 1% sequences
SEQS=`grep -c ">" ../data/processed_files/hg19_genes.split.30mer.fa | awk '{print int(($1*0.01)+0.5)}'`
! pyfastx sample -n ${SEQS} -s 42 --sequential-read -o ../data/processed_files/hg19_genes.split.30mer.1perc.fa  ../data/processed_files/hg19_genes.split.30mer.fa

# get fasta sequences
! awk '{print ">" $1 "\n" $NF}' ../data/processed_files/hg19_mc5.bed > ../data/processed_files/hg19_mc5.fa
! awk '{print ">" $1 "\n" $NF}' ../data/processed_files/hg19_mc3.bed > ../data/processed_files/hg19_mc3.fa

# plot logo
! for i in ../data/processed_files/hg19_mc*fa; do \
    mkdir -p ${i%.*} && \
    kpLogo $i -bgfile ../data/processed_files/hg19_genes.split.30mer.1perc.fa -o ${i%.*}/kpLogo; done